In [1]:
pwd

'/root/bert-classify'

In [1]:
import catboost

In [ ]:
import pandas as pd

train = pd.read_csv('train.csv',sep='|')
# test  = pd.read_csv('guang_dataset/test.csv')
train

import re
import jieba as jb
import numpy as np
import pandas as pd
from catboost import CatBoostClassifier

def remove_punctuation(line):
    line = str(line)
    if line.strip()=='':
        return ''
    rule = re.compile(u"[^a-zA-Z0-9\u4E00-\u9FA5]")
    line = rule.sub('',line)
    return line

train_df = train[['content','ret']]

train_df['clean_text'] = train_df['content'].apply(remove_punctuation)
train_df['cut_text'] = train_df['clean_text'].apply(lambda x:" ".join([w for w in list(jb.cut(x))]))

y = train_df['ret']
X_train=np.array(train_df['cut_text'].astype('str'))

model = CatBoostClassifier(task_type='GPU')
model.fit(X_train,y,text_features=[0])

# X_train[:2]



In [ ]:
label = []
for i in range(test.shape[0]):
    label.append(model.predict([test[i]]))

In [8]:
test = pd.read_csv('test_dataset.csv',sep='|')
test_df = test


test_df['clean_text'] = test_df['content'].apply(remove_punctuation)
test_df['cut_text'] = train_df['clean_text'].apply(lambda x: " ".join([w for w in list(jb.cut(x))]))

X_test = np.array(test_df['cut_text'].astype('str'))
# X_test = np.array(test_df['cut_text'].apply(lambda x:" ".join([w for w in list(jb.cut(x))])))

In [9]:
id2label = {0: '非常感谢您向我们提供这些宝贵信息，我们已经对您反应的问题进行记录反馈，同时我公司也会持续优化网络，建议您留意以后的使用情况，谢谢！;投诉现象最近发生时间：一直信号不好;',
 1: '非常感谢您向我们提供这些宝贵信息，我们已经对您反应的问题进行记录反馈，同时我公司也会持续优化网络，建议您留意以后的使用情况，谢谢！;投诉现象最近发生时间：以前信号很好，最近信号不好;',
 2: '非常感谢您向我们提供这些宝贵信息，我们已经对您反应的问题进行记录反馈，同时我公司也会持续优化网络，建议您留意以后的使用情况，谢谢！;投诉现象最近发生时间：不清楚（包含刚搬过来或者路过等情况）;',
 3: '客户您好，您投诉的区域的相关基站已经建设完成，目前正在对设备紧某某人调试中，希望您能耐心等待，感谢您对我们工作的支持。;投诉现象最近发生时间：一直信号不好;',
 4: '您反映的问题,经初步核查为周边用户私装非官方干扰器或直放站对我公司现网基站设备产生干扰导致，我公司无法强制要求用户拆除，此问题将反馈至北京市无线电管理部门，建议您关注后续使用情况，感谢您的理解与支持。;投诉现象最近发生时间：以前信号很好，最近信号不好;',
 5: '客户来电反映无法接收到短信，请提供下发的号码******以及时间**:**:**，为客户记录反映，客户的诉求****',
 6: '您好，您反映的问题我们已经记录，由于周边基站出现故障，导致该地址周边信号覆盖较差，我公司已安排进行故障处理。”（如用户强烈不认可解答的时限，则解答为因该问题解决涉及到较多客观原因，非我公司可控，完成时间初步估计以“失效时间”字段内容为准）。;投诉现象最近发生时间：以前信号很好，最近信号不好;',
 7: '【仅适用于用户上网慢现象】您反映的问题,经初步核查为周边用户私装非官方干扰器,直放站对我公司现网基站设备产生干扰导致，我公司无法强制要求用户拆除，此问题将反馈至北京市无线电管理部门，建议您关注后续使用情况，感谢您的理解与支持。;投诉现象最近发生时间：以前信号很好，最近信号不好;',
 8: '用户来电反映对202101月份的流量使用情况有疑义，我某某人常按照上网日志解释并建议其查询上网详单，用户强烈不认可，要求我方为其核实具体使用情况，请尽快处理，谢谢！',
 9: '用户来电反映手机在所有地方均无法连接网络，查询用户手机状态正常，已为用户一键体检，无效；建议用户换机尝试，用户不认可，要求投诉，请尽快处理，谢谢。',
 10: '客户来电反映从今天开始，在所有位置信号正常，无法上网，换机换卡无效，经系统查询号码状态正常，已为客户进行“一键体检”操作无效，客户要求我方核实具体原因并回复，谢谢！',
 11: '您好，您反映的问题我们已经记录，***覆盖较差，我公司已提出相应的解决方案，会争取尽快解决，请您继续留意使用情况。（如用户询问时限，可告知：因涉及到较多客观原因，暂时无法确认完工及开通时间，但是请您放心，我司有专门的技术部门进行后续跟踪处理。如用户强烈不认可并追问时限，可按GIS信息中预计完成时间告知用户，因该问题解决涉及到较多客观原因，非我公司可控，完成时间可能会提前或滞后，但是请您放心，我公司会有专门的技术部门全力跟踪支撑）。;投诉现象最近发生时间：一直信号不好;',
 12: '“您好，您反映的问题我们已经记录，由于周边基站出现故障，导致该地址周边信号覆盖较差，我公司已安排进行故障处理。”（如用户强烈不认可并追问时限，则按【预计完成时间】字段解答，因该问题解决涉及到较多客观原因，非我公司可控，完成时间可能会提前或滞后，但是请您放心，我公司会有专门的技术部门全力跟踪支撑）。;投诉现象最近发生时间：以前信号很好，最近信号不好;',
 13: '用户来电反映手机在所有地方均无法连接网络，查询用户手机状态正常，已为用户一键体检，无效；建议用户换机尝试，用户不认可，要求投诉，请尽快处理，谢谢。\t\t',
 14: '用户来电反映手机信号正常，无法上网，我方查询用户状态正常，请处理，谢谢',
 15: '您反映的问题我们已经记录。您投诉的区域，已有基站建设计划，我们正在获取资源，因涉及在别人地方上加资源首先要得到当地居民（或业主）、物业的允许,目前方案正在制定，仍需一段时间才能开工建设，希望您能耐心等待。如果上述资源全部协调顺利，就能完成建站，如果您有什么物业关系，能够加快相关资源协调力度，也希望您拨打10010和我们联系，我们自身也会加大协调力度，尽早解决您反映的问题。;投诉现象最近发生时间：一直信号不好;',
 16: '您好，您反映的问题我们已经记录，***覆盖较差，我公司已提出相应的解决方案，会争取尽快解决，请您继续留意使用情况。（如用户询问时限，可告知：因涉及到较多客观原因，暂时无法确认完工及开通时间，但是请您放心，我司有专门的技术部门进行后续跟踪处理。如用户强烈不认可并追问时限，可按GIS信息中预计完成时间告知用户，因该问题解决涉及到较多客观原因，非我公司可控，完成时间可能会提前或滞后，但是请您放心，我公司会有专门的技术部门全力跟踪支撑）。;投诉现象最近发生时间：以前信号很好，最近信号不好;',
 17: '客户来电反映无法接收到短信，请提供下发的号码******以及时间**:**:**，为客户记录反映，客户的诉求*******',
 18: '客服中心接到客户反映通过通信大数据行程卡获取验证码后，点击查询提示“验证码错误”，再次重新获取时无法接收到验证码。',
 19: '客户您好！您反映的问题我们已经记录。您投诉的区域，已有基站建设计划，我们正在获取资源，因涉及在别人地方上加资源首先要得到当地居民（或业主）、物业的允许,目前方案正在制定，仍需一段时间才能开工建设，希望您能耐心等待。如果上述资源全部协调顺利，最长需要三个月才能完成建站，如果您有什么物业关系，能够加快相关资源协调力度，也希望您拨打10139和我们联系，我们自身也会加大协调力度，尽早解决您反映的问题。;投诉现象最近发生时间：一直信号不好;',
 20: '【仅适用于用户上网慢现象】您反映的问题,经初步核查为周边用户私装非官方干扰器,直放站对我公司现网基站设备产生干扰导致，我公司无法强制要求用户拆除，此问题将反馈至北京市无线电管理部门，建议您关注后续使用情况，感谢您的理解与支持。;投诉现象最近发生时间：一直信号不好;',
 21: '用户来电反映手机在所有地方均无法连接网络，查询用户手机状态正常，已为用户一键体检，无效；建议用户换机尝试，用户不认可，要求投诉，请尽快处理，谢谢。！',
 22: '客户您好！您反映的问题我们已经记录。您投诉的区域，已有基站建设计划，我们正在获取资源，因涉及在别人地方上加资源首先要得到当地居民（或业主）、物业的允许,目前方案正在制定，仍需一段时间才能开工建设，希望您能耐心等待。如果上述资源全部协调顺利，最长需要三个月才能完成建站，如果您有什么物业关系，能够加快相关资源协调力度，也希望您拨打10139和我们联系，我们自身也会加大协调力度，尽早解决您反映的问题。;投诉现象最近发生时间：以前信号很好，最近信号不好;',
 23: '客户来电反映在所有位置信号正常，无法上网，换机换卡无效，经系统查询号码上网功能及状态均正常，已为客户进行“一键体检”操作无效，客户要求我方核实具体原因并回复，谢谢！\t\t',
 24: '用户来电反映手机无法正常使用，信号正常，在所有位置均无法正常使用，经我方系统查询用户状态正常，已通过一键体检正常，建议已尝试重启手机换卡换手机。用户不认可。现要求我方核实处理并回复，请处理谢谢！',
 25: '用户来电反映对202012月份的流量使用情况有疑义，我某某人常按照上网日志解释并建议其查询上网详单，用户强烈不认可，要求我方为其核实具体使用情况，请尽快处理，谢谢！',
 26: '“您好，您反映的问题我们已经记录，由于周边基站出现故障，导致该地址周边信号覆盖较差，我公司已安排进行故障处理。”\t（如用户强烈不认可并追问时限，则按【预计完成时间】字段解答，因该问题解决涉及到较多客观原因，非我公司可控，完成时间可能会提前或滞后，但是请您放心，我公司会有专门的技术部门全力跟踪支撑）。;投诉现象最近发生时间：以前信号很好，最近信号不好;',
 27: '客户您好，您投诉的区域的相关基站已经建设完成，目前正在对设备紧某某人调试中，希望您能耐心等待，感谢您对我们工作的支持。;投诉现象最近发生时间：不清楚（包含刚搬过来或者路过等情况）;',
 28: '您反映的问题,经初步核查为周边用户私装非官方干扰器对我公司现网基站设备产生干扰导致，我公司无法强制要求用户拆除，此问题将反馈至北京市无线电管理部门，建议您关注后续使用情况，感谢您的理解与支持。;投诉现象最近发生时间：以前信号很好，最近信号不好;',
 29: '客服中心接到客户反映通过通信大数据行程卡获取验证码后，提示“应用服务异常”。客户诉求：*******',
 30: '客户办理日期：\t\t办理渠道（选择手厅/短信）：\t\t客户诉求：\t\t客户态度（解释认可/解释不认可）：\t\t备注：解释认可的选择工单办结，对于不认可的选择工单提交。\t\t',
 31: '“您好，您反映的问题我们已经记录，由于周边基站出现故障，导致该地址周边信号覆盖较差，我公司已安排进行故障处理。”\t\t（如用户强烈不认可并追问时限，则按【预计完成时间】字段解答，因该问题解决涉及到较多客观原因，非我公司可控，完成时间可能会提前或滞后，但是请您放心，我公司会有专门的技术部门全力跟踪支撑）。;投诉现象最近发生时间：以前信号很好，最近信号不好;',
 32: '您反映的问题,我公司已经进行实地测试，经核实您的问题是由于政府行为引***覆盖较差，我公司已提出相应的解决方案，会争取尽快解决，请您继续留意使用情况。;投诉现象最近发生时间：以前信号很好，最近信号不好;',
 33: '您反映的问题,经初步核查为周边用户私装非官方干扰器或直放站对我公司现网基站设备产生干扰导致，我公司无法强制要求用户拆除，此问题将反馈至北京市无线电管理部门，建议您关注后续使用情况，感谢您的理解与支持。;投诉现象最近发生时间：不清楚（包含刚搬过来或者路过等情况）;',
 34: '用户来电对本月产生的流量使用情况不认可，我方按照移动上网日志向用户解释，用户不认可，要求我方专人核实处理并回复，解释无效，转接专席失败，请尽快处理，谢谢！',
 35: '用户来电反映对202102月份的流量使用情况有疑义，我某某人常按照上网日志解释并建议其查询上网详单，用户强烈不认可，要求我方为其核实具体使用情况，请尽快处理，谢谢！',
 36: '客户来电反映其不欠费，但现却接到催缴欠费短信，经系统查询为几年前的历史欠费记录，请先安抚客户此问题已上报，并建议客户随时关注，如客户强烈不认可，记录工单反馈。',
 37: '用户反映，最近号码无法上网，网络特别慢 在所以位置都无法使用，通过我方刷新没有效果，要求我方上报核实原因提供解决方案，有升级投诉倾向，用户着急烦请核实处理',
 38: '用户来电反映手机在所有地方均无法连接网络，查询用户手机状态正常，已为用户一键体检，无效；建议用户换机尝试，用户不认可，要求投诉，请尽快处理，谢谢。！\t\t\t\t\t\t\t',
 39: '客户来电反映从202101时间开始，在所有位置信号正常，无法上网，换机换卡无效，经系统查询号码状态正常，已为客户进行“一键体检”操作无效，客户要求我方核实具体原因并回复，谢谢！'}






In [11]:
label = []
for i in range(X_test.shape[0]):
    label.append(model.predict([X_test[i]]))

test['preds'] = 

In [15]:
test['ret'] = [id2label[i[0]] for i in label]

In [16]:
test

,id,content,clean_text,cut_text,preds,ret
0,25001,【坐席】您好，实习话务员为您服务，【客户】，唉，你好那个，我刚收到咱们那个话费通知的短信了，...,坐席您好实习话务员为您服务客户唉你好那个我刚收到咱们那个话费通知的短信了我看了一下说我上个月...,坐席 您好 高兴 为您服务 客户 唉 你好 那个 我 家里 这个 不太好 坐席 信号 不好 ...,非常感谢您向我们提供这些宝贵信息，我们已经对您反应的问题进行记录反馈，同时我公司也会持续优化...,非常感谢您向我们提供这些宝贵信息，我们已经对您反应的问题进行记录反馈，同时我公司也会持续优化...
1,25002,【坐席】您好，很高兴为您服务，【客户】，嗯，你好，那个先说一下那个不太好意思，因为今天不是我...,坐席您好很高兴为您服务客户嗯你好那个先说一下那个不太好意思因为今天不是我因为之前投诉不是你们...,坐席 很 高兴 为您服务 您好 客户 唉 你 是 联通 吗 坐席 嗯 对 这里 是 北京 联...,客服中心接到客户反映通过通信大数据行程卡获取验证码后，点击查询提示“验证码错误”，再次重新获...,客服中心接到客户反映通过通信大数据行程卡获取验证码后，点击查询提示“验证码错误”，再次重新获...
2,25003,【坐席】您好，很高兴为您服务，您好，【客户】，那个我这个套餐到期了也没人通知我呀，我原来是1...,坐席您好很高兴为您服务您好客户那个我这个套餐到期了也没人通知我呀我原来是138块6的现在它我...,坐席 很 高兴 为您服务 您好 客户 唉 你好 能 听到 我 说话 吗 坐席 可以 听到 您...,非常感谢您向我们提供这些宝贵信息，我们已经对您反应的问题进行记录反馈，同时我公司也会持续优化...,非常感谢您向我们提供这些宝贵信息，我们已经对您反应的问题进行记录反馈，同时我公司也会持续优化...
3,25004,【坐席】很高兴为您服务，您好？【客户】，唉，你好，我想问一下我那个话费现在欠了多少了？【坐席...,坐席很高兴为您服务您好客户唉你好我想问一下我那个话费现在欠了多少了坐席来电这号码欠费了多少钱...,坐席 很 高兴 为您服务 客户 喂 你好 我 现在 在 屋里 边 联通 信号 打电话 怎么 ...,非常感谢您向我们提供这些宝贵信息，我们已经对您反应的问题进行记录反馈，同时我公司也会持续优化...,非常感谢您向我们提供这些宝贵信息，我们已经对您反应的问题进行记录反馈，同时我公司也会持续优化...
4,25005,【坐席】您好，很高兴为您服务，【客户】，唉，你好，你帮我查一下，我有一个业务是返返钱的业务，...,坐席您好很高兴为您服务客户唉你好你帮我查一下我有一个业务是返返钱的业务一放有的1月份的时候给...,坐席 您好 很 高兴 为您服务 喂 您好 唉 你好 客户 因为 我 这 现在 就是 那个 网...,非常感谢您向我们提供这些宝贵信息，我们已经对您反应的问题进行记录反馈，同时我公司也会持续优化...,非常感谢您向我们提供这些宝贵信息，我们已经对您反应的问题进行记录反馈，同时我公司也会持续优化...
...,...,...,...,...,...,...
7521,32522,【坐席】您好很高兴为您服务【客户】唉你好那个是这样的问题我呀以前是每月十九元套餐嗯在一月份的...,坐席您好很高兴为您服务客户唉你好那个是这样的问题我呀以前是每月十九元套餐嗯在一月份的一月份你...,NaN,非常感谢您向我们提供这些宝贵信息，我们已经对您反应的问题进行记录反馈，同时我公司也会持续优化...,非常感谢您向我们提供这些宝贵信息，我们已经对您反应的问题进行记录反馈，同时我公司也会持续优化...
7522,32523,【坐席】您好，很高兴为您服务，【客户】，喂，您好，就是就是我们这那个查看联通的信号好差呀，联...,坐席您好很高兴为您服务客户喂您好就是就是我们这那个查看联通的信号好差呀联通信号不好对呀然后你...,NaN,非常感谢您向我们提供这些宝贵信息，我们已经对您反应的问题进行记录反馈，同时我公司也会持续优化...,非常感谢您向我们提供这些宝贵信息，我们已经对您反应的问题进行记录反馈，同时我公司也会持续优化...
7523,32524,【坐席】您好，高兴为您服务？【客户】，唉，你好，我之前那个打过那个电话，然后就是这个手机已经...,坐席您好高兴为您服务客户唉你好我之前那个打过那个电话然后就是这个手机已经停机好久了然后你们每...,NaN,非常感谢您向我们提供这些宝贵信息，我们已经对您反应的问题进行记录反馈，同时我公司也会持续优化...,非常感谢您向我们提供这些宝贵信息，我们已经对您反应的问题进行记录反馈，同时我公司也会持续优化...
7524,32525,【坐席】您好，实习话务员为您服务？【客户】，嗯，你好，我想在我们家装一个那个光纤宽带，可以帮...,坐席您好实习话务员为您服务客户嗯你好我想在我们家装一个那个光纤宽带可以帮我查一下这个地址能装...,NaN,非常感谢您向我们提供这些宝贵信息，我们已经对您反应的问题进行记录反馈，同时我公司也会持续优化...,非常感谢您向我们提供这些宝贵信息，我们已经对您反应的问题进行记录反馈，同时我公司也会持续优化...


In [17]:
test = test[['id','ret']]
test

,id,ret
0,25001,非常感谢您向我们提供这些宝贵信息，我们已经对您反应的问题进行记录反馈，同时我公司也会持续优化...
1,25002,客服中心接到客户反映通过通信大数据行程卡获取验证码后，点击查询提示“验证码错误”，再次重新获...
2,25003,非常感谢您向我们提供这些宝贵信息，我们已经对您反应的问题进行记录反馈，同时我公司也会持续优化...
3,25004,非常感谢您向我们提供这些宝贵信息，我们已经对您反应的问题进行记录反馈，同时我公司也会持续优化...
4,25005,非常感谢您向我们提供这些宝贵信息，我们已经对您反应的问题进行记录反馈，同时我公司也会持续优化...
...,...,...
7521,32522,非常感谢您向我们提供这些宝贵信息，我们已经对您反应的问题进行记录反馈，同时我公司也会持续优化...
7522,32523,非常感谢您向我们提供这些宝贵信息，我们已经对您反应的问题进行记录反馈，同时我公司也会持续优化...
7523,32524,非常感谢您向我们提供这些宝贵信息，我们已经对您反应的问题进行记录反馈，同时我公司也会持续优化...
7524,32525,非常感谢您向我们提供这些宝贵信息，我们已经对您反应的问题进行记录反馈，同时我公司也会持续优化...


In [18]:
test['ret'].value_counts()

非常感谢您向我们提供这些宝贵信息，我们已经对您反应的问题进行记录反馈，同时我公司也会持续优化网络，建议您留意以后的使用情况，谢谢！;投诉现象最近发生时间：以前信号很好，最近信号不好;                                                                                                                                                            5268
非常感谢您向我们提供这些宝贵信息，我们已经对您反应的问题进行记录反馈，同时我公司也会持续优化网络，建议您留意以后的使用情况，谢谢！;投诉现象最近发生时间：一直信号不好;                                                                                                                                                                   1185
客户您好，您投诉的区域的相关基站已经建设完成，目前正在对设备紧某某人调试中，希望您能耐心等待，感谢您对我们工作的支持。;投诉现象最近发生时间：一直信号不好;                                                                                                                                                                          149
非常感谢您向我们提供这些宝贵信息，我们已经对您反应的问题进行记录反馈，同时我公司也会持续优化网络，建议您留意以后的使用情况，谢谢！;投诉现象最近发生时间：不清楚（包含刚搬过来或者路过等情况）;                                                                                                                                                    

In [20]:
test.to_csv('sub.csv',sep='|')

In [ ]:
label = []
for i in range(X_train.shape[0]):
    label.append(model.predict([X_train[i]]))

train['preds'] = label
train

print(train.labels.value_counts())
print(train.preds.value_counts())

from sklearn.metrics import classification_report,f1_score

print("macro_f1:",f1_score(train['labels'], train['preds'], average='macro'))
print(classification_report(train['labels'], train['preds'],digits=4))

test

test_df = test[['text','labels']]
test_df['clean_text'] = test_df['text'].apply(remove_punctuation)
test_df['cut_text'] = train_df['clean_text'].apply(lambda x: " ".join([w for w in list(jb.cut(x))]))
X_test = np.array(test_df['cut_text'].apply(lambda x:" ".join([w for w in list(jb.cut(x))])))


label = []
for i in range(X_test.shape[0]):
    label.append(model.predict([X_test[i]]))

test['preds'] = label


from sklearn.metrics import classification_report,f1_score

print("macro_f1:",f1_score(test['labels'], test['preds'], average='macro'))
print(classification_report(test['labels'], test['preds'],digits=4))

In [22]:
train = pd.read_csv('train.csv',sep='|')
train['ret'].value_counts()

0     1180
1     1171
2      152
3      149
4       64
5       62
6       42
7       41
8       36
9       35
10      28
12      27
11      27
13      25
14      24
15      22
16      22
17      21
18      19
19      17
21      16
20      16
23      15
22      15
29      14
27      14
30      14
28      14
26      14
24      14
25      14
31      13
32      13
34      12
36      12
33      12
35      12
39      11
37      11
38      11
Name: ret, dtype: int64

In [25]:
train_df

,text,labels
0,【坐席】您好，高兴为您服务，【客户】，唉，你好那个，我家里这个***不太好，【坐席】信号不好...,1
1,【坐席】很高兴为您服务，您好？【客户】，唉，你是联通吗【坐席】嗯，对，这里是北京联通客服中心...,18
2,【坐席】很高兴为您服务，您好？【客户】，唉，你好，能听到我说话吗，【坐席】可以听到您说，【客...,0
3,【坐席】很高兴为您服务，【客户】，喂，你好，我现在在屋里边联通信号，打电话怎么老是断，就是经...,1
4,【坐席】您好，很高兴为您服务，喂，您好，唉你好【客户】，因为我这现在就是那个网，好像特别差，...,0
...,...,...
3426,【坐席】您好，很高兴为您服务【客户】，我家这块，我这打不打的微信电话就往，【坐席】喂你好，【...,28
3427,【坐席】高兴为您服务您好【客户】那我想问一下为什么四G三的网络差【坐席】噢上网比较差是吗【客...,1
3428,【客户】您好，很高兴为您服务【坐席】，唉，您好，是这样子的，就是我们现在是，【客户】在北京市...,0
3429,【客户】唉，你好，我我想问一下你们那个联通的，在这个通州某某人家湾这块有流量那个那个，嗯信号...,0


In [31]:
train_df['labels'] = train_df['labels'].apply(lambda x:x+1)

In [32]:
train_df

,text,labels
0,【坐席】您好，高兴为您服务，【客户】，唉，你好那个，我家里这个***不太好，【坐席】信号不好...,2
1,【坐席】很高兴为您服务，您好？【客户】，唉，你是联通吗【坐席】嗯，对，这里是北京联通客服中心...,19
2,【坐席】很高兴为您服务，您好？【客户】，唉，你好，能听到我说话吗，【坐席】可以听到您说，【客...,1
3,【坐席】很高兴为您服务，【客户】，喂，你好，我现在在屋里边联通信号，打电话怎么老是断，就是经...,2
4,【坐席】您好，很高兴为您服务，喂，您好，唉你好【客户】，因为我这现在就是那个网，好像特别差，...,1
...,...,...
3426,【坐席】您好，很高兴为您服务【客户】，我家这块，我这打不打的微信电话就往，【坐席】喂你好，【...,29
3427,【坐席】高兴为您服务您好【客户】那我想问一下为什么四G三的网络差【坐席】噢上网比较差是吗【客...,2
3428,【客户】您好，很高兴为您服务【坐席】，唉，您好，是这样子的，就是我们现在是，【客户】在北京市...,1
3429,【客户】唉，你好，我我想问一下你们那个联通的，在这个通州某某人家湾这块有流量那个那个，嗯信号...,1


In [36]:
import pandas as pd
from simpletransformers.classification import ClassificationModel
# train_df  = pd.read_csv('train.csv',sep='|')
# train_df
# train_df['text'] = train_df['content']
# train_df['labels'] = train_df['ret']
# train_df = train_df[['text','labels']]


train = train_df
# train=train_df[500:]
# test = train_df[:500]

# train = pd.read_csv('raw_data/train.csv').rename(columns={'label':'labels'})
# test = pd.read_csv('raw_data/test.csv')
# test  = pd.read_csv('guang_dataset/test_1.csv',names=['text','labels'])
# test = pd.DataFrame()
# test['text'] = train['review'][:500]
# test['labels'] = train['label'][:500]
model = ClassificationModel('bert', '../../ccf-aqiy/bert-model/', num_labels=40, args={'reprocess_input_data': True, 'overwrite_output_dir': True, 'learning_rate':5e-5, 'train_batch_size': 32, 'max_seq_length': 512, 'num_train_epochs': 1})
model.train_model(train)
# predictions, _ = model.predict([row['text'] for _, row in test.iterrows()])

Some weights of the model checkpoint at ../../ccf-aqiy/bert-model/ were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [35]:
rm -rf 

In [62]:
test = pd.read_csv('test_dataset.csv',sep='|')

In [63]:
test['text'] = test['content']
test=test[['id','text']]

In [65]:
test

,id,text
0,25001,【坐席】您好，实习话务员为您服务，【客户】，唉，你好那个，我刚收到咱们那个话费通知的短信了，...
1,25002,【坐席】您好，很高兴为您服务，【客户】，嗯，你好，那个先说一下那个不太好意思，因为今天不是我...
2,25003,【坐席】您好，很高兴为您服务，您好，【客户】，那个我这个套餐到期了也没人通知我呀，我原来是1...
3,25004,【坐席】很高兴为您服务，您好？【客户】，唉，你好，我想问一下我那个话费现在欠了多少了？【坐席...
4,25005,【坐席】您好，很高兴为您服务，【客户】，唉，你好，你帮我查一下，我有一个业务是返返钱的业务，...
...,...,...
7521,32522,【坐席】您好很高兴为您服务【客户】唉你好那个是这样的问题我呀以前是每月十九元套餐嗯在一月份的...
7522,32523,【坐席】您好，很高兴为您服务，【客户】，喂，您好，就是就是我们这那个查看联通的信号好差呀，联...
7523,32524,【坐席】您好，高兴为您服务？【客户】，唉，你好，我之前那个打过那个电话，然后就是这个手机已经...
7524,32525,【坐席】您好，实习话务员为您服务？【客户】，嗯，你好，我想在我们家装一个那个光纤宽带，可以帮...


In [64]:
test.to_csv('test_ccf_kefu.csv',sep=',',index=False)


In [43]:
!head sub.csv

|id|ret
0|25001|非常感谢您向我们提供这些宝贵信息，我们已经对您反应的问题进行记录反馈，同时我公司也会持续优化网络，建议您留意以后的使用情况，谢谢！;投诉现象最近发生时间：以前信号很好，最近信号不好;
1|25002|客服中心接到客户反映通过通信大数据行程卡获取验证码后，点击查询提示“验证码错误”，再次重新获取时无法接收到验证码。
2|25003|非常感谢您向我们提供这些宝贵信息，我们已经对您反应的问题进行记录反馈，同时我公司也会持续优化网络，建议您留意以后的使用情况，谢谢！;投诉现象最近发生时间：一直信号不好;
3|25004|非常感谢您向我们提供这些宝贵信息，我们已经对您反应的问题进行记录反馈，同时我公司也会持续优化网络，建议您留意以后的使用情况，谢谢！;投诉现象最近发生时间：以前信号很好，最近信号不好;
4|25005|非常感谢您向我们提供这些宝贵信息，我们已经对您反应的问题进行记录反馈，同时我公司也会持续优化网络，建议您留意以后的使用情况，谢谢！;投诉现象最近发生时间：一直信号不好;
5|25006|非常感谢您向我们提供这些宝贵信息，我们已经对您反应的问题进行记录反馈，同时我公司也会持续优化网络，建议您留意以后的使用情况，谢谢！;投诉现象最近发生时间：以前信号很好，最近信号不好;
6|25007|非常感谢您向我们提供这些宝贵信息，我们已经对您反应的问题进行记录反馈，同时我公司也会持续优化网络，建议您留意以后的使用情况，谢谢！;投诉现象最近发生时间：以前信号很好，最近信号不好;
7|25008|非常感谢您向我们提供这些宝贵信息，我们已经对您反应的问题进行记录反馈，同时我公司也会持续优化网络，建议您留意以后的使用情况，谢谢！;投诉现象最近发生时间：一直信号不好;
8|25009|非常感谢您向我们提供这些宝贵信息，我们已经对您反应的问题进行记录反馈，同时我公司也会持续优化网络，建议您留意以后的使用情况，谢谢！;投诉现象最近发生时间：以前信号很好，最近信号不好;


In [44]:
!head submission.csv

id,label
0,1
1,18
2,0
3,1
4,1
5,1
6,1
7,0
8,1


In [91]:
sub = pd.read_csv('submission.csv')
sub

,id,label
0,0,35
1,1,3
2,2,3
3,3,2
4,4,24
...,...,...
7521,7521,25
7522,7522,1
7523,7523,3
7524,7524,0


In [92]:
sub['ret'] = sub['label'].apply(lambda x:id2label[x])

sub

,id,label,ret
0,0,35,用户来电反映对202102月份的流量使用情况有疑义，我某某人常按照上网日志解释并建议其查询上...
1,1,3,客户您好，您投诉的区域的相关基站已经建设完成，目前正在对设备紧某某人调试中，希望您能耐心等待...
2,2,3,客户您好，您投诉的区域的相关基站已经建设完成，目前正在对设备紧某某人调试中，希望您能耐心等待...
3,3,2,非常感谢您向我们提供这些宝贵信息，我们已经对您反应的问题进行记录反馈，同时我公司也会持续优化...
4,4,24,用户来电反映手机无法正常使用，信号正常，在所有位置均无法正常使用，经我方系统查询用户状态正常...
...,...,...,...
7521,7521,25,用户来电反映对202012月份的流量使用情况有疑义，我某某人常按照上网日志解释并建议其查询上...
7522,7522,1,非常感谢您向我们提供这些宝贵信息，我们已经对您反应的问题进行记录反馈，同时我公司也会持续优化...
7523,7523,3,客户您好，您投诉的区域的相关基站已经建设完成，目前正在对设备紧某某人调试中，希望您能耐心等待...
7524,7524,0,非常感谢您向我们提供这些宝贵信息，我们已经对您反应的问题进行记录反馈，同时我公司也会持续优化...


In [73]:
sub['ret'] = sub['label'].apply(lambda x:id2label[x])

sub

,id,label,ret
0,0,25,用户来电反映对202012月份的流量使用情况有疑义，我某某人常按照上网日志解释并建议其查询上...
1,1,0,非常感谢您向我们提供这些宝贵信息，我们已经对您反应的问题进行记录反馈，同时我公司也会持续优化...
2,2,0,非常感谢您向我们提供这些宝贵信息，我们已经对您反应的问题进行记录反馈，同时我公司也会持续优化...
3,3,1,非常感谢您向我们提供这些宝贵信息，我们已经对您反应的问题进行记录反馈，同时我公司也会持续优化...
4,4,0,非常感谢您向我们提供这些宝贵信息，我们已经对您反应的问题进行记录反馈，同时我公司也会持续优化...
...,...,...,...
7521,7521,0,非常感谢您向我们提供这些宝贵信息，我们已经对您反应的问题进行记录反馈，同时我公司也会持续优化...
7522,7522,1,非常感谢您向我们提供这些宝贵信息，我们已经对您反应的问题进行记录反馈，同时我公司也会持续优化...
7523,7523,3,客户您好，您投诉的区域的相关基站已经建设完成，目前正在对设备紧某某人调试中，希望您能耐心等待...
7524,7524,0,非常感谢您向我们提供这些宝贵信息，我们已经对您反应的问题进行记录反馈，同时我公司也会持续优化...


In [93]:
sub = sub[['id','ret']]

In [94]:
sub

,id,ret
0,0,用户来电反映对202102月份的流量使用情况有疑义，我某某人常按照上网日志解释并建议其查询上...
1,1,客户您好，您投诉的区域的相关基站已经建设完成，目前正在对设备紧某某人调试中，希望您能耐心等待...
2,2,客户您好，您投诉的区域的相关基站已经建设完成，目前正在对设备紧某某人调试中，希望您能耐心等待...
3,3,非常感谢您向我们提供这些宝贵信息，我们已经对您反应的问题进行记录反馈，同时我公司也会持续优化...
4,4,用户来电反映手机无法正常使用，信号正常，在所有位置均无法正常使用，经我方系统查询用户状态正常...
...,...,...
7521,7521,用户来电反映对202012月份的流量使用情况有疑义，我某某人常按照上网日志解释并建议其查询上...
7522,7522,非常感谢您向我们提供这些宝贵信息，我们已经对您反应的问题进行记录反馈，同时我公司也会持续优化...
7523,7523,客户您好，您投诉的区域的相关基站已经建设完成，目前正在对设备紧某某人调试中，希望您能耐心等待...
7524,7524,非常感谢您向我们提供这些宝贵信息，我们已经对您反应的问题进行记录反馈，同时我公司也会持续优化...


In [95]:
sub['id']= sub['id'].apply(lambda x:x+25001)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [96]:
sub

,id,ret
0,25001,用户来电反映对202102月份的流量使用情况有疑义，我某某人常按照上网日志解释并建议其查询上...
1,25002,客户您好，您投诉的区域的相关基站已经建设完成，目前正在对设备紧某某人调试中，希望您能耐心等待...
2,25003,客户您好，您投诉的区域的相关基站已经建设完成，目前正在对设备紧某某人调试中，希望您能耐心等待...
3,25004,非常感谢您向我们提供这些宝贵信息，我们已经对您反应的问题进行记录反馈，同时我公司也会持续优化...
4,25005,用户来电反映手机无法正常使用，信号正常，在所有位置均无法正常使用，经我方系统查询用户状态正常...
...,...,...
7521,32522,用户来电反映对202012月份的流量使用情况有疑义，我某某人常按照上网日志解释并建议其查询上...
7522,32523,非常感谢您向我们提供这些宝贵信息，我们已经对您反应的问题进行记录反馈，同时我公司也会持续优化...
7523,32524,客户您好，您投诉的区域的相关基站已经建设完成，目前正在对设备紧某某人调试中，希望您能耐心等待...
7524,32525,非常感谢您向我们提供这些宝贵信息，我们已经对您反应的问题进行记录反馈，同时我公司也会持续优化...


In [97]:
ss = pd.read_csv('_144.csv',sep='|')
ss

,id,ret
0,25001,用户来电对202101月产生的15.93元流量费不认可，我方经系统查询安上网日志向用户解释，...
1,25002,********某某人用户来电对11月和12月产生的流量费用50.00元不认可，称当时办理的...
2,25003,用户来电反映自己号码********在2021-01-01 00: 00: 00 到期，我方...
3,25004,用户来电反映自己号码欠费-175.75，我方查询用户号码欠费-175.75，用户称自己没有使...
4,25005,用户来电反映之前问题，详见工单流水号：ts********，用户来电反映之前有一个返还90元...
...,...,...
7521,32522,客户来电反映本机号码办理套餐时告知每月19元5g流量，客户表示当时办理时告知每月19元5g流...
7522,32523,非常感谢您向我们提供这些宝贵信息，我们已经对您反应的问题进行记录反馈，同时我公司也会持续优化...
7523,32524,********用户来电反映，2021年1月办理了国内闲时流量包10元3gb（立即生效），称...
7524,32525,用户来电反映自己在家里安装宽带，但是安装师傅上门查看并没有光纤覆盖，用户要求我方核实，请处理...


In [56]:
!wc -l test_ccf_kefu.csv

7527 test_ccf_kefu.csv


In [84]:
xx = ['非常感谢您向我们提供这些宝贵信息，我们已经对您反应的问题进行记录反馈，同时我公司也会持续优化网络，建议您留意以后的使用情况，谢谢！;投诉现象最近发生时间：一直信号不好;']
xx_2 =['非常感谢您向我们提供这些宝贵信息，我们已经对您反应的问题进行记录反馈，同时我公司也会持续优化网络，建议您留意以后的使用情况，谢谢！;投诉现象最近发生时间：以前信号很好，最近信号不好;']

In [86]:
ss_in_xx = ss[ss.ret.isin(xx)]
ss_in_xx_2=ss[ss.ret.isin(xx_2)]

In [87]:
ss_in_xx_id = ss_in_xx['id'].tolist()
ss_in_xx_2_id = ss_in_xx_2['id'].tolist()

In [98]:
for i in range(len(sub)):
    if sub['id'][i] in ss_in_xx_id:
        sub['ret'][i] = '非常感谢您向我们提供这些宝贵信息，我们已经对您反应的问题进行记录反馈，同时我公司也会持续优化网络，建议您留意以后的使用情况，谢谢！;投诉现象最近发生时间：一直信号不好;'
    elif sub['id'][i] in ss_in_xx_2_id:
        sub['ret'][i] = '非常感谢您向我们提供这些宝贵信息，我们已经对您反应的问题进行记录反馈，同时我公司也会持续优化网络，建议您留意以后的使用情况，谢谢！;投诉现象最近发生时间：以前信号很好，最近信号不好;'

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3331: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

In [99]:
sub

,id,ret
0,25001,用户来电反映对202102月份的流量使用情况有疑义，我某某人常按照上网日志解释并建议其查询上...
1,25002,客户您好，您投诉的区域的相关基站已经建设完成，目前正在对设备紧某某人调试中，希望您能耐心等待...
2,25003,客户您好，您投诉的区域的相关基站已经建设完成，目前正在对设备紧某某人调试中，希望您能耐心等待...
3,25004,非常感谢您向我们提供这些宝贵信息，我们已经对您反应的问题进行记录反馈，同时我公司也会持续优化...
4,25005,用户来电反映手机无法正常使用，信号正常，在所有位置均无法正常使用，经我方系统查询用户状态正常...
...,...,...
7521,32522,用户来电反映对202012月份的流量使用情况有疑义，我某某人常按照上网日志解释并建议其查询上...
7522,32523,非常感谢您向我们提供这些宝贵信息，我们已经对您反应的问题进行记录反馈，同时我公司也会持续优化...
7523,32524,客户您好，您投诉的区域的相关基站已经建设完成，目前正在对设备紧某某人调试中，希望您能耐心等待...
7524,32525,非常感谢您向我们提供这些宝贵信息，我们已经对您反应的问题进行记录反馈，同时我公司也会持续优化...


In [100]:
sub.to_csv('submul.csv',sep='|')